<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

# **Predict Length of Hospital Stay**
In this notebook, we will go through the process of loading data into a <a href="https://docs.microsoft.com/sql/big-data-cluster/big-data-cluster-overview?view=sqlallproducts-allversions">SQL Server 2019 Big Data Cluster</a> and then train machine learning models on the data.
Follow the **Prerequisites** section below before starting the rest of the notebook.
Once you have a Big Data Cluster set up, continue to the next steps:
1. Prepare Data
2. Define Training Features
3. Train a Random Forest and a Neural Network (MLP) model
4. Store Models in SQL Server and then run Native Predict

<!-- <span style="color:red"><font size="3">Please press the "Run Cells" button to run the notebook</font></span> -->

## **Prerequisites**
* Connect to a SQL Server 2019 Big Data Cluster. <a href="https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-get-started?view=sqlallproducts-allversions">Get Started with SQL Server Big Data Cluster</a> to deploy and connect to a Big Data Cluster.
* Upload the hospital length of stay dataset into HDFS at `user/LengthOfStay.csv`.
* Create or select a database to use for this notebook.

## **1. Prepare Data**
In this section, we will:
* Create an External Data Source and Table to our data in HDFS.
* Split the data into train/test sets.

Create an external data source in your **target** database.

In [0]:
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
  CREATE EXTERNAL DATA SOURCE SqlStoragePool
  WITH (LOCATION = 'sqlhdfs://controller-svc/default');
END

Create an External File Format that will be used to parse the LengthOfStay.csv.

In [3]:
CREATE EXTERNAL FILE FORMAT [FileFormat_dbo_patients-data]
            WITH (FORMAT_TYPE = DELIMITEDTEXT, FORMAT_OPTIONS (FIELD_TERMINATOR = ',', STRING_DELIMITER = '"', FIRST_ROW = 2));

Create an external table, dbo.patients, for the LengthOfStay data in HDFS.

In [6]:
CREATE EXTERNAL TABLE [dbo].[patients](
	eid int NOT NULL,
	vdate date NOT NULL,
	rcount varchar(2) NOT NULL,
	gender char(1) NOT NULL,
	dialysisrenalendstage int NOT NULL,
	asthma int NOT NULL,
	irondef int NOT NULL,
	pneum int NOT NULL,
	substancedependence int NOT NULL,
	psychologicaldisordermajor int NOT NULL,
	depress int NOT NULL,
	psychother int NOT NULL,
	fibrosisandother int NOT NULL,
	malnutrition int NOT NULL,
	hemo int NOT NULL,
	hematocrit float NOT NULL,
	neutrophils float NOT NULL,
	sodium float NOT NULL,
	glucose float NOT NULL,
	bloodureanitro float NOT NULL,
	creatinine float NOT NULL,
	bmi float NOT NULL,
	pulse float NOT NULL,
	respiration float NOT NULL,
	secondarydiagnosisnonicd9 int NOT NULL,
	discharged date NOT NULL,
	facid char(1) NOT NULL,
	lengthofstay int NOT NULL
)
WITH (LOCATION = N'/user/LengthOfStay.csv', DATA_SOURCE = [SqlStoragePool], FILE_FORMAT = [FileFormat_dbo_patients-data]);

Commands completed successfully.

Total execution time: 00:00:00.153

### **Split Data into Train/Test sets**

In [12]:
CREATE TABLE dbo.Train_Id(
	eid int NOT NULL,
	INDEX cci_train_ids CLUSTERED COLUMNSTORE
);
GO

DECLARE @train_percent int = 70;
TRUNCATE TABLE Train_Id;
INSERT INTO Train_Id (eid)
SELECT eid
FROM dbo.patients
WHERE ABS(CAST(BINARY_CHECKSUM(eid, NEWID()) as int)) % 100 < @train_percent;
GO

-- Training set
SELECT  (hematocrit - AVG(hematocrit) OVER())/STDEV(hematocrit) OVER()  as hematocrit,
		(neutrophils - AVG(neutrophils) OVER())/STDEV(neutrophils) OVER() as neutrophils,
		(sodium - AVG(sodium) OVER())/STDEV(sodium) OVER() as sodium,
		(glucose - AVG(glucose) OVER())/STDEV(glucose) OVER()  as glucose,
		(bloodureanitro - AVG(bloodureanitro) OVER())/STDEV(bloodureanitro) OVER() as bloodureanitro,
		(creatinine - AVG(creatinine) OVER())/STDEV(creatinine) OVER() as creatinine,
		(bmi - AVG(bmi) OVER())/STDEV(bmi) OVER() as bmi,
		(pulse - AVG(pulse) OVER())/STDEV(pulse) OVER() as pulse,
		(respiration - AVG(respiration) OVER())/STDEV(respiration) OVER() as respiration,
		CAST(hemo as int) + CAST(dialysisrenalendstage as int) + CAST(asthma as int) + CAST(irondef as int) + CAST(pneum as int) +
			CAST(substancedependence as int) + CAST(psychologicaldisordermajor as int) + CAST(depress as int) +
            CAST(psychother as int) + CAST(fibrosisandother as int) + CAST(malnutrition as int) AS number_of_issues,
		asthma, 
		depress, 
		dialysisrenalendstage, 
		fibrosisandother, 
		(case when gender = 'M' then 1 else 0 end) as gender, 
		hemo, 
		irondef, 
		malnutrition, 
		pneum, 
		psychologicaldisordermajor, 
		psychother,
		secondarydiagnosisnonicd9, 
		substancedependence, 
		(case when rcount = '5+' then 5 else rcount end) as rcount, 
		lengthofstay
INTO dbo.patients_train
FROM dbo.patients as l
WHERE EXISTS(SELECT * FROM Train_Id as t WHERE t.eid = l.eid);
GO

SELECT  (hematocrit - AVG(hematocrit) OVER())/STDEV(hematocrit) OVER() as hematocrit,
		(neutrophils - AVG(neutrophils) OVER())/STDEV(neutrophils) OVER() as neutrophils,
		(sodium - AVG(sodium) OVER())/STDEV(sodium) OVER() as sodium,
		(glucose - AVG(glucose) OVER())/STDEV(glucose) OVER()  as glucose,
		(bloodureanitro - AVG(bloodureanitro) OVER())/STDEV(bloodureanitro) OVER() as bloodureanitro,
		(creatinine - AVG(creatinine) OVER())/STDEV(creatinine) OVER() as creatinine,
		(bmi - AVG(bmi) OVER())/STDEV(bmi) OVER() as bmi,
		(pulse - AVG(pulse) OVER())/STDEV(pulse) OVER() as pulse,
		(respiration - AVG(respiration) OVER())/STDEV(respiration) OVER() as respiration,
		CAST(hemo as int) + CAST(dialysisrenalendstage as int) + CAST(asthma as int) + CAST(irondef as int) + CAST(pneum as int) +
			CAST(substancedependence as int) + CAST(psychologicaldisordermajor as int) + CAST(depress as int) +
            CAST(psychother as int) + CAST(fibrosisandother as int) + CAST(malnutrition as int) AS number_of_issues,
		asthma, 
		depress, 
		dialysisrenalendstage, 
		fibrosisandother, 
		(case when gender = 'M' then 1 else 0 end) as gender, 
		hemo, 
		irondef,
		malnutrition, 
		pneum, 
		psychologicaldisordermajor, 
		psychother,
		secondarydiagnosisnonicd9, 
		substancedependence, 
		(case when rcount = '5+' then 5 else rcount end) as rcount, 
		lengthofstay
INTO dbo.patients_score
FROM dbo.patients  as l
WHERE NOT EXISTS(SELECT * FROM Train_Id as t WHERE t.eid = l.eid);
GO

SELECT TOP(30)
CAST([hematocrit] AS REAL) AS [hematocrit],
CAST([neutrophils] AS REAL) AS [neutrophils],
CAST([sodium] AS REAL) AS [sodium],
CAST([glucose] AS REAL) AS [glucose],
CAST([bloodureanitro] AS REAL) AS [bloodureanitro],
CAST([creatinine] AS REAL) AS [creatinine],
CAST([bmi] AS REAL) AS [bmi],
CAST([pulse] AS REAL) AS [pulse],
CAST([respiration] AS REAL) AS [respiration],
CAST([number_of_issues] AS BIGINT) AS [number_of_issues],
CAST([asthma] AS BIGINT) AS [asthma],
CAST([depress] AS BIGINT) AS [depress],
CAST([dialysisrenalendstage] AS BIGINT) AS [dialysisrenalendstage],
CAST([fibrosisandother] AS BIGINT) AS [fibrosisandother],
CAST([gender] AS BIGINT) AS [gender],
CAST([hemo] AS BIGINT) AS [hemo],
CAST([irondef] AS BIGINT) AS [irondef],
CAST([malnutrition] AS BIGINT) AS [malnutrition],
CAST([pneum] AS BIGINT) AS [pneum],
CAST([psychologicaldisordermajor] AS BIGINT) AS [psychologicaldisordermajor],
CAST([psychother] AS BIGINT) AS [psychother],
CAST([secondarydiagnosisnonicd9] AS BIGINT) AS [secondarydiagnosisnonicd9],
CAST([substancedependence] AS BIGINT) AS [substancedependence],
CAST([rcount] AS BIGINT) AS [rcount]
INTO dbo.patients_score_cooked
FROM dbo.patients_score
GO

## **Change Kernel from SQL to Python 3**

## **2. Define Training Features**

### **Import Libraries**

In [0]:
import numpy as np
import pandas as pd
import pyodbc
import pickleimport pickle

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, RobustScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

### **Define Training Features**

In [0]:
patient_columns = ['hematocrit', 'neutrophils','sodium', 'glucose', 'bloodureanitro', 'creatinine', 'bmi', 
                   'pulse', 'respiration', 'number_of_issues', 'asthma', 'depress', 'dialysisrenalendstage', 
                   'fibrosisandother', 'gender', 'hemo', 'irondef', 'malnutrition', 'pneum', 'psychologicaldisordermajor', 
                   'psychother', 'secondarydiagnosisnonicd9', 'substancedependence', 'rcount', 'lengthofstay']

continuous_features  = ['hematocrit', 'neutrophils','sodium', 'glucose', 'bloodureanitro', 'creatinine', 'bmi'
                        , 'pulse', 'respiration']

categorical_features = ['number_of_issues', 'asthma', 'depress', 'dialysisrenalendstage', 'fibrosisandother', 'gender', 
                        'hemo', 'irondef', 'malnutrition', 'pneum', 'psychologicaldisordermajor', 'psychother', 
                        'secondarydiagnosisnonicd9', 'substancedependence', 'rcount']

sql_conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0}; SERVER=localhost; DATABASE=sonnx; Uid=sa; Pwd=Cisl_2014;')

data_query = "SELECT [hematocrit],[neutrophils],[sodium],[glucose],[bloodureanitro],[creatinine],[bmi],[pulse], \
                     [respiration],[number_of_issues],[asthma],[depress],[dialysisrenalendstage],[fibrosisandother], \
                     [gender],[hemo],[irondef],[malnutrition],[pneum],[psychologicaldisordermajor],[psychother], \
                     [secondarydiagnosisnonicd9],[substancedependence],[rcount],[lengthofstay] \
              FROM [dbo].[patients_train]"

def load_data_from_sqlserver(sql_conn):
    data = pd.read_sql(data_query, sql_conn)
    #data = pd.read_csv(path, header=0, names=patient_columns)
    data = data[continuous_features + categorical_features + ['lengthofstay']]
    # print(data['lengthofstay'].tail())
    data_y = data['lengthofstay'];
    data_x = data.loc[:, data.columns != 'lengthofstay'];
    return data_x, data_y

def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        # print(df.columns);
        # print(df.dtypes);
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([1, 1])
        elif v == 'float64':
            t = FloatTensorType([1, 1])
        else:
            t = StringTensorType([1, 1])
        inputs.append((k, t))
    return inputs

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(sparse=False))])

preprocessor = ColumnTransformer(
     transformers=[
         ('continuous', continuous_transformer, [0, 1, 2, 3, 4, 5, 6, 7, 8]),
         ('categorical', categorical_transformer, [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
     ])

## **3. Train Random Forest and Neural Network model**
In this section, we will train two models: Random Forest and a Neural Network (Multi-Layer Perceptron).
As defined earlier, these models are trained using a 70/30 train/test split.

We will then save them to a pickle file to later be stored into SQL Server.



### **Random Forest** 

In [0]:
def train_patients_rf():
    train_x, train_y = load_data_from_sqlserver(sql_conn)
    
    model = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(max_depth = 4))])

    model.fit(train_x, train_y)
    return model

trained_rf_model = train_patients_rf()

### **Neural Network (Multi-Layer Perceptron)**

In [0]:
def train_patients_mlp():
    train_x, train_y = load_data_from_sqlserver(sql_conn)
    
    model = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=100))])

    model.fit(train_x, train_y)
    return model

trained_mlp_model = train_patients_mlp()



### **Serialize models using Pickle**
In the cell below, define a filename for each model you want to save. Later, you will access this file to retrieve the models.

In [0]:
filehandler_rf = open('', 'w') # create .pkl file in directory
pickle.dump(trained_rf_model, filehandler_rf)

filehandler_mlp = open('', 'w') # create .pkl file in directory
pickle.dump(trained_mlp_model, filehandler_mlp)

## **Switch to SQL Kernel**

## **4. Store Models in SQL Server and then run Native Predict**

### **Create table and then store the model in the table**
You will have to insert the path to the model files you created above.

In [0]:
DROP TABLE IF EXISTS dbo.hospital_stay_models;
GO
CREATE TABLE [dbo].[hospital_stay_models](
	[Id] [int] IDENTITY(1,1) NOT NULL,
	[Data] [varbinary](max) NULL,
	[Description] [varchar](200) NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO

In [0]:
INSERT INTO hospital_stay_models (Data, Description) 
	SELECT BulkColumn, 'Random Forest model' FROM OPENROWSET (BULK '', SINGLE_BLOB) AS X -- Insert path to rf model file

INSERT INTO hospital_stay_models (Data, Description) 
	SELECT BulkColumn, 'MLP model' FROM OPENROWSET (BULK '', SINGLE_BLOB) AS X -- Insert path to mlp model file


### **Native Predict Using Stored Models**

In [0]:
DECLARE @model varbinary(max) = (SELECT Data from hospital_stay_models where id = 1)
SELECT p.output_label AS 'length of stay'
FROM PREDICT(model=@model, data=[patients_score_cooked]) with(output_label bigint) AS p

In [0]:
DECLARE @model varbinary(max) = (SELECT Data FROM hospital_stay_models WHERE id = 2)
SELECT p.output_label AS 'length of stay', patients_score_cooked.*
FROM PREDICT(model=@model, data=patients_score_cooked) with(output_label bigint) AS p